In [4]:
# @title
### EP2 MAC0417 / MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
# DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
# #
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
# DESONESTIDADE ACADÊMICA. #
# #
# #
# Nome : # Mikhail Futorny
# NUSP : # 5258765
# Turma: # MAC0417
# Prof.: # Ronaldo Fumio Hashimoto
##################################################################

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
augmented_dataset_folder = './content/augmentedDataSet'
metadata_path = './content/Metadados/metadados.csv'
normalized_dataset_folder = './content/normalizedDataset'

metadata = pd.read_csv(metadata_path) #Leitura dos metadados
if not os.path.exists(normalized_dataset_folder):
    os.makedirs(normalized_dataset_folder)

NameError: name 'pd' is not defined

In [ ]:
def histogram_equalization(image):
    hist, bins = np.histogram(image.flatten(), bins=256, range=(0, 255), density=True)

    acumulada = hist.cumsum()
    acumulada_normalizada = (acumulada - acumulada.min()) / (acumulada.max() - acumulada.min())  # Normalização da acumulada

    # Equalização
    equalized_image = np.interp(image.flatten(), bins[:-1], acumulada_normalizada)
    equalized_image = equalized_image.reshape(image.shape)

    return equalized_image

In [ ]:
def plot_histogram(image, title):
    plt.hist(image.ravel(), bins=256, range=(0, 1), alpha=0.7, color='b', density=True)
    plt.title(title)
    plt.xlabel('Intensidade')
    plt.ylabel('Frequencia normalizada')
    plt.xlim([0, 1])
    plt.grid(True)
    plt.tight_layout()

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = plt.imread(img_path)
        if img is not None:
            images.append((filename, img))
    return images

In [ ]:
def analyze_datasets(dataset_folder):
    mean_histograms_before = []
    mean_histograms_after = []
    images_before_after = []

    for filename in os.listdir(dataset_folder): #Fazemos equalizacao para cada imagem no dataset
        img_path = os.path.join(dataset_folder, filename)
        img = plt.imread(img_path)


        img_equalized = histogram_equalization(img)

        hist_before, _ = np.histogram(img.flatten(), bins=256, range=(0,256))
        hist_after, _ = np.histogram(img_equalized.flatten(), bins=256, range=(0,256))

        mean_histograms_before.append(hist_before)
        mean_histograms_after.append(hist_after)

        images_before_after.append((img, img_equalized))

    mean_hist_before = np.mean(mean_histograms_before, axis=0)
    mean_hist_after = np.mean(mean_histograms_after, axis=0)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(mean_hist_before)
    plt.title('Histograma médio antes da normalização')
    plt.xlabel('Intensidade Pixel')
    plt.ylabel('Frequencia')

    plt.subplot(1, 2, 2)
    plt.plot(mean_hist_after)
    plt.title('Histograma médio depois da normalização')
    plt.xlabel('Intensidade Pixel')
    plt.ylabel('Frequencia')

    plt.tight_layout()
    plt.show()

    num_samples = min(len(images_before_after), 5)  # Mostramos 5 imagens de exemplo
    plt.figure(figsize=(15, 6))
    for i in range(num_samples):
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(images_before_after[i][0], cmap='gray')
        plt.title('Antes')
        plt.axis('off')

        plt.subplot(2, num_samples, i + num_samples + 1)
        plt.imshow(images_before_after[i][1], cmap='gray')
        plt.title('Depois')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
if not os.path.exists(normalized_dataset_folder):
    os.makedirs(normalized_dataset_folder)

for subdir in os.listdir(augmented_dataset_folder):
    subdir_path = os.path.join(augmented_dataset_folder, subdir)
    if os.path.isdir(subdir_path):
        normalized_subdir_path = os.path.join(normalized_dataset_folder, subdir)
        if not os.path.exists(normalized_subdir_path):
            os.makedirs(normalized_subdir_path)

        for filename in os.listdir(subdir_path):
            if filename.endswith('.jpeg'):
                img_path = os.path.join(subdir_path, filename)
                img = plt.imread(img_path)
                img_equalized = histogram_equalization(img)
                save_path = os.path.join(normalized_subdir_path, filename)
                plt.imsave(save_path, img_equalized, cmap='gray')

In [ ]:
print('Normal:')
analyze_datasets('./content/augmentedDataSet/Normal')
print('Contraste:')
analyze_datasets('./content/augmentedDataSet/Contraste')
print('Exponencial:')
analyze_datasets('./content/augmentedDataSet/Exponencial')
print('Laplaciano:')
analyze_datasets('./content/augmentedDataSet/Laplaciano')
print('MeanFilter:')
analyze_datasets('./content/augmentedDataSet/MeanFilter')
print('Logaritmo:')
analyze_datasets('./content/augmentedDataSet/Logaritmo')

Conclusão: O histograma médio após normalização jogou muito peso para os pixeis de valor zero, inicialmente estranho. Entretanto, dado o caráter preto e branco da imagem é um comportamento válido possível. Em relação às transformações, o augmented dataset no geral melhorou o contraste entre os objetos, sendo os de logaritmo e exponencial os melhores, provavelmente graças à escolha apropriada dos parâmetros, que é adaptada ao escuro, o que resulta em uma ótima performance para o dataset das imagens escuras, em relação às outras transformações. Em relação à normalização, o impacto mais significativo foi para o Laplaciano, que gerou a melhor visibilidade após a normalização. Para as outras transformações também foi positivo, porém com menor impacto. Percebe-se que seu impacto é bom tanto em casos escuros e claros. Em suma, podemos concluir que as transformações implementadas tem impacto positivo na visualização do Dataset.